Figure des essaies à supprimer

- saccades dans GAP (début GAP - fin GAP + 150ms) > 45ms
- trop de saccade dans poursuite (>= 1/3 de la poursuite)

In [1]:
%load_ext autoreload
%autoreload 2
from __future__ import division, print_function, absolute_import
import pickle
import os
import matplotlib.pyplot as plt
import sys
sys.path.append('..')
from ANEMO import read_edf
import numpy as np
from lmfit import  Model, Parameters
%matplotlib inline

In [2]:
def exponentiel (x, tau, maxi, start_anti, a_anti, latency, bino) :
    a_anti = a_anti/1000
    time = np.arange(len(x))
    vitesse = []
    for t in range(len(time)):
        if start_anti >= latency :
            if time[t] < latency :
                vitesse.append(0)
            else :
                vitesse.append((bino*2-1)*maxi*(1-np.exp(-1/tau*(time[t]-latency))))
        else :
            if time[t] < start_anti :
                vitesse.append(0)
            else :
                if time[t] < latency :
                    vitesse.append((time[t]-start_anti)*a_anti)
                    x = (time[t]-start_anti)*a_anti
                else :
                    vitesse.append((bino*2-1)*maxi*(1-np.exp(-1/tau*(time[t]-latency)))+x)
    return vitesse

In [3]:
!ls ../data/*.asc

../data/eyeMvt_s01_2017-10-23_100057.asc
../data/eyeMvt_s02_2017-09-26_095637.asc
../data/eyeMvt_s03_2017-10-03_134421.asc
../data/eyeMvt_s04_2017-09-27_161040.asc
../data/eyeMvt_s05_2017-10-03_143803.asc
../data/eyeMvt_s06_2017-09-28_115250.asc
../data/eyeMvt_s07_2017-09-20_151043.asc
../data/eyeMvt_s08_2017-10-26_121823.asc
../data/eyeMvt_s09_2017-11-08_094717.asc
../data/eyeMvt_s10_2017-11-16_153313.asc
../data/eyeMvt_s11_2017-11-08_150410.asc
../data/eyeMvt_s12_2017-11-17_172706.asc


In [4]:
sujet = ['s01','s02','s03','s04','s05','s06','s07','s08','s09','s10','s11', 's12']
time = ['2017-10-23_100057','2017-09-26_095637',
        '2017-10-03_134421','2017-09-27_161040',
        '2017-10-03_143803','2017-09-28_115250',
        '2017-09-20_151043','2017-10-26_121823',
        '2017-11-08_094717','2017-11-16_153313',
        '2017-11-08_150410','2017-11-17_172706']

In [5]:
Liste_suppresion_essais = {}

for x in range(len(sujet)) :
    timeStr = time[x]
    observer = sujet[x]
    mode = 'eyeMvt'
    
    print(observer,timeStr)
    
    file = os.path.join('..','data', mode + '_' + observer + '_' + timeStr + '.pkl')
    with open(file, 'rb') as fichier :
        exp = pickle.load(fichier, encoding='latin1')

    resultats = os.path.join('..','data', mode + '_' + observer + '_' + timeStr + '.asc')
    data = read_edf(resultats, 'TRIALID')

    N_trials = exp['N_trials']
    N_blocks = exp['N_blocks']
    p = exp['p']

    Liste_suppresion_essais['%s'%observer] = []
    for block in range(N_blocks) :
        supp_trial = []
        for trial in range(N_trials) :
            bino=p[trial, block, 0]
            trial_data = trial + N_trials*block
            data_x = data[trial_data]['x']
            trackertime = data[trial_data]['trackertime']

            StimulusOf = data[trial_data]['events']['msg'][14][0]
            TargetOn = data[trial_data]['events']['msg'][15][0]
            TargetOff = data[trial_data]['events']['msg'][16][0]
            saccades = data[trial_data]['events']['Esac']

            trackertime_0 = data[trial_data]['trackertime'][0]

            gradient_x = np.gradient(data_x)
            gradient_deg = gradient_x * 1/exp['px_per_deg'] * 1000 # gradient en deg/sec

            ##################################################
            # SUPPRESSION DES SACCADES
            ##################################################
            gradient_deg_NAN = gradient_deg
            s_gap= []
            for s in range(len(saccades)) :
                s_gap.append([])
                if saccades[s][1]-trackertime_0+15 <= (len(trackertime)) : #(TargetOff-trackertime_0-300) : #
                    for x_data in np.arange((saccades[s][0]-trackertime_0-5), (saccades[s][1]-trackertime_0+15)) :
                        gradient_deg_NAN[x_data] = np.nan
                        #if x_data in np.arange(StimulusOf-trackertime_0, TargetOn-trackertime_0+150) :
                        if x_data in np.arange(TargetOn-trackertime_0-150, TargetOn-trackertime_0+150) :
                            s_gap[s].append(x_data)
                '''else :
                    for x_data in np.arange((saccades[s][0]-trackertime_0-5), (len(trackertime))) :
                        gradient_deg_NAN[x_data] = np.nan
                        #if x_data in np.arange(StimulusOf-trackertime_0, TargetOn-trackertime_0+150) :
                        if x_data in np.arange(TargetOn-trackertime_0-150, TargetOn-trackertime_0+150) :
                            s_gap[s].append(x_data)'''
            ##################################################

            # supp grosse saccade GAP
            t = None
            for x in range(len(s_gap)) :
                if len(s_gap[x]) > 45 :
                    if t is None :
                        #print('GAP', block, trial)
                        supp_trial.append(trial)
                        t = True
            if t is None :
                # supp trop saccade poursuite
                lala2 = gradient_deg_NAN[TargetOn-trackertime_0:TargetOff-trackertime_0-300]
                baba2 = [x for x in lala2 if str(x)=='nan']
                if len(baba2) >= len(lala2)/3.6 :
                    #print('poursuite', block, trial)
                    supp_trial.append(trial)
                    '''for s in range(len(saccades)) :
                        if saccades[s][0]-trackertime_0-5 > TargetOn-trackertime_0+350 :
                            print('sacc',((saccades[s][1]-trackertime_0+15))-(saccades[s][0]-trackertime_0-5))'''
                    

        
        print(block, supp_trial)
        Liste_suppresion_essais['%s'%observer].extend([supp_trial])

s01 2017-10-23_100057
0 []
1 []
2 []
s02 2017-09-26_095637
0 [76, 156]
1 [57, 66, 71, 126, 128, 129, 137]
2 [79, 137]
s03 2017-10-03_134421
0 []
1 []
2 [95]
s04 2017-09-27_161040
0 [101, 120, 185]
1 [49, 66, 85, 127, 130, 144, 164]
2 [31, 36]
s05 2017-10-03_143803
0 [14, 29, 35, 50, 58, 59, 60, 71, 73, 77, 80, 104, 157, 161, 167, 173, 174, 175, 178, 198]
1 [12, 31, 64, 68, 69, 70, 96, 98, 121, 129]
2 [100, 158, 177]
s06 2017-09-28_115250
0 []
1 []
2 []
s07 2017-09-20_151043
0 [10, 42, 82, 83, 143, 150, 151, 163, 179]
1 [72, 116, 123, 162, 164, 178]
2 [5, 11, 21, 32, 44, 53, 98]
s08 2017-10-26_121823
0 [28, 32, 38, 40, 46, 52, 68, 71, 76, 81, 85, 86, 89, 90, 98, 122, 126, 130, 131, 140, 143, 154, 166, 179, 183, 196, 199]
1 [36, 48, 57, 94, 96, 145, 149, 182, 185, 194, 197]
2 [1, 26, 44, 52, 84, 95, 97, 131, 137, 140, 148, 157, 168, 173, 177, 186, 187]
s09 2017-11-08_094717
0 []
1 [30, 180]
2 [85, 90, 170, 177, 190]
s10 2017-11-16_153313
0 [50, 56, 96, 137, 143, 148]
1 [21, 40]
2 [186, 1